In [111]:
import pickle
import gradio as gr
import pandas as pd

with open('linear.sav', 'rb') as f_in:
    model_lin = pickle.load(f_in)

def transform_features(effectiveness, sideEffect):
    effects = {'Highly Effective': '5', 
                                  'Considerably Effective': '4',
                                  'Moderately Effective': '3',
                                  'Marginally Effective': '2',
                                  'Ineffective': '1'}
    for answer, score in effects.items():
        effectiveness = effectiveness.replace(answer, score)

    sides = {'No Side Effects': '1',
                               'Mild Side Effects': '2',
                               'Moderate Side Effects': '3',
                               'Extremely Severe Side Effects': '5',
                               'Severe Side Effects': '4',
                               }

    for answer, score in sides.items():
        sideEffect = sideEffect.replace(answer, score)

    revs_df = pd.DataFrame([(int(effectiveness), int(sideEffect))], columns=('effectiveness', 'sideEffects'))
    return revs_df

def compute_rating(effectiveness, sideEffect):
    rev = transform_features(effectiveness, sideEffect)
    result = round(model_lin.predict(rev)[0])
    
    if result<=0:
        result=1

    return result

demo = gr.Interface(fn=compute_rating, 
                            inputs=[gr.Radio(['Ineffective', 'Marginally Effective', 'Moderately Effective', 'Considerably Effective', 'Highly Effective']), 
                                    gr.Radio(['No Side Effects','Mild Side Effects','Moderate Side Effects','Severe Side Effects','Extremely Severe Side Effects'])], 
                            outputs=gr.outputs.Textbox(label='Predicted Drug Rating'), title='drug rating calculator', description='Please enter drug name as "drug_name"',
                            examples=[['Highly Effective', 'No Side Effects'],['Ineffective', 'Severe Side Effects'] ])

demo.launch()

/opt/anaconda3/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7882/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fca6b40dee0>, 'http://127.0.0.1:7882/', None)

In [2]:
import gradio as gr
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import pandas as pd
import pickle
from nltk.corpus import stopwords


ENGLISH_STOP_WORDS = set( stopwords.words('english') ).union(set(ENGLISH_STOP_WORDS))
ENGLISH_STOP_WORDS.remove('no')
ENGLISH_STOP_WORDS.remove('not')
ENGLISH_STOP_WORDS.remove('less')
ENGLISH_STOP_WORDS.remove('more')
ENGLISH_STOP_WORDS.add('mg')

stop_words= ENGLISH_STOP_WORDS

In [3]:
with open('logregression.sav', 'rb') as f_in:
    model_log = pickle.load(f_in)

model_log.feature_names_in_

def get_bag_of_words(txt):
    txt = txt.replace(r'\d+','')
    txt = [txt]
    vec = CountVectorizer(ngram_range=(1, 3), stop_words=stop_words, max_features=10000).fit(txt)        
    bag_of_words = vec.transform(txt)                                                                #tuple of numbers (0, 2980) 1
    return vec, bag_of_words  

def create_bow_df(bowt):             
    vec, count_matrix = bowt                     # allReviews only
    count_array = count_matrix.toarray()
    df_bow = pd.DataFrame(data=count_array, columns=vec.get_feature_names())
    return df_bow

def get_tf_idf(bow, tf_or_idf=False):
    bow_df = create_bow_df(bow)
    vect = TfidfTransformer(use_idf=tf_or_idf).fit(bow_df)
    tf_matrix = vect.transform(bow_df) #df_bow
    tf_array = tf_matrix.toarray()
    df_tfx = pd.DataFrame(data=tf_array, columns=vect.get_feature_names_out())
    return df_tfx
    
def transform_features(effectiveness, sideEffect, review):
    effects = {'Highly Effective': '5', 
                                  'Considerably Effective': '4',
                                  'Moderately Effective': '3',
                                  'Marginally Effective': '2',
                                  'Ineffective': '1'}
    for answer, score in effects.items():
        effectiveness = effectiveness.replace(answer, score)

    sides = {'No Side Effects': '1',
                               'Mild Side Effects': '2',
                               'Moderate Side Effects': '3',
                               'Extremely Severe Side Effects': '5',
                               'Severe Side Effects': '4',
                               }
    for answer, score in sides.items():
        sideEffect = sideEffect.replace(answer, score)

    rev_df = get_bag_of_words(review)
    rev_df = get_tf_idf(rev_df)

    rev_df['effectiveness_x'] = int(effectiveness)
    rev_df['sideEffects'] = int(sideEffect)

    features = model_log.feature_names_in_
    feature_dict = {k:0 for k in features}              #fits to model's words

    rev_dict = rev_df.to_dict()

    for col in rev_df.columns:
        test = feature_dict.get(col, None)
        if test is not None:
            feature_dict[col] = rev_dict[col]
    review_tf = pd.DataFrame(feature_dict, index=[0])
    
    return review_tf

def compute_rating(effectiveness, sideEffect, review):
    rev = transform_features(effectiveness, sideEffect, review)
    result = model_log.predict(rev)
    if result == 0:
        result = 'Low Rating'
    elif result == 1:
        result = 'High Rating'
    return result

demo = gr.Interface(fn=compute_rating, 
                            inputs=[gr.Radio(['Ineffective', 'Marginally Effective', 'Moderately Effective', 'Considerably Effective', 'Highly Effective']), 
                                    gr.Radio(['No Side Effects','Mild Side Effects','Moderate Side Effects','Severe Side Effects','Extremely Severe Side Effects']),
                                    'textbox'],
                            outputs=gr.outputs.Textbox(label='Predicted Drug Rating'), title='drug rating calculator', description='Uses LogRegression model, and multigram term frequency. \n Please enter drug name as "drug_name"',
                            examples=[['Highly Effective', 'No Side Effects', '	drug_name, an angeotensin receptor blocker, lowered my blood pressure without side effects'],['Ineffective', 'Severe Side Effects', 'frequent urination, always thristy, dizziness'] ])

demo.launch()

/opt/anaconda3/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fbe29901640>, 'http://127.0.0.1:7860/', None)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
with open('knnregressor_19.sav', 'rb') as f_in:
    model_knr = pickle.load(f_in)

model_knr.feature_names_in_

def get_bag_of_words(txt):
    txt = txt.replace(r'\d+','')
    txt = [txt]
    vec = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words, max_features=10000).fit(txt)        
    bag_of_words = vec.transform(txt)                                                                #tuple of numbers (0, 2980) 1
    return vec, bag_of_words  

def create_bow_df(bowt):             
    vec, count_matrix = bowt                     # allReviews only
    count_array = count_matrix.toarray()
    df_bow = pd.DataFrame(data=count_array, columns=vec.get_feature_names())
    return df_bow

def get_tf_idf(bow, tf_or_idf=True):
    bow_df = create_bow_df(bow)
    vect = TfidfTransformer(use_idf=tf_or_idf).fit(bow_df)
    tf_matrix = vect.transform(bow_df) #df_bow
    tf_array = tf_matrix.toarray()
    df_tfx = pd.DataFrame(data=tf_array, columns=vect.get_feature_names_out())
    return df_tfx
    
def transform_features(effectiveness, sideEffect, review):
    effects = {'Highly Effective': '5', 
                                  'Considerably Effective': '4',
                                  'Moderately Effective': '3',
                                  'Marginally Effective': '2',
                                  'Ineffective': '1'}
    for answer, score in effects.items():
        effectiveness = effectiveness.replace(answer, score)

    sides = {'No Side Effects': '1',
                               'Mild Side Effects': '2',
                               'Moderate Side Effects': '3',
                               'Extremely Severe Side Effects': '5',
                               'Severe Side Effects': '4',
                               }
    for answer, score in sides.items():
        sideEffect = sideEffect.replace(answer, score)

    rev_df = get_bag_of_words(review)
    rev_df = get_tf_idf(rev_df)

    rev_df['effectiveness_x'] = int(effectiveness)
    rev_df['sideEffects'] = int(sideEffect)

    features = model_log.feature_names_in_
    feature_dict = {k:0 for k in features}              #fits to model's words

    rev_dict = rev_df.to_dict()

    for col in rev_df.columns:
        test = feature_dict.get(col, None)
        if test is not None:
            feature_dict[col] = rev_dict[col]
    review_tf = pd.DataFrame(feature_dict, index=[0])
    
    return review_tf

def compute_rating(effectiveness, sideEffect, review):
    rev = transform_features(effectiveness, sideEffect, review)
    result = round(model_knr.predict(rev)[0])
    
    if result<=0:
        result=1

    return result

demo = gr.Interface(fn=compute_rating, 
                            inputs=[gr.Radio(['Ineffective', 'Marginally Effective', 'Moderately Effective', 'Considerably Effective', 'Highly Effective']), 
                                    gr.Radio(['No Side Effects','Mild Side Effects','Moderate Side Effects','Severe Side Effects','Extremely Severe Side Effects']),
                                    'textbox'],
                            outputs=gr.outputs.Textbox(label='Predicted Drug Rating'), title='drug rating calculator', description='Uses KNRegressor model, and bigram inverse document frequency. \n Please enter drug name as "drug_name"',
                            examples=[['Highly Effective', 'No Side Effects', '	drug_name, an angeotensin receptor blocker, lowered my blood pressure without side effects'],['Ineffective', 'Severe Side Effects', 'frequent urination, always thristy, dizziness'] ])

demo.launch()

/opt/anaconda3/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fbe10425d90>, 'http://127.0.0.1:7862/', None)